<a href="https://colab.research.google.com/github/francisco-perez-sorrosal/miniai/blob/main/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random, os
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

def seeder(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seeder(42)

In [ ]:
batch_size_train = 64
batch_size_test = 1000

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

In [ ]:
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
def display_digits_grid(images, grid_size, example_targets=None):
  n_grid_rows, n_grid_cols = grid_size

  # Select images to present
  num_images_to_present = n_grid_rows * n_grid_cols
  images = images[:num_images_to_present]

  # Create a figure with subplots
  fig, axs = plt.subplots(n_grid_rows, n_grid_cols, figsize=(n_grid_cols, n_grid_rows),
                          subplot_kw={'xticks': [], 'yticks': []})

  axs = axs.flat  # Flatten the array of axes

  # Plot each image in its respective subplot
  for i, ax in enumerate(axs):
    ax.imshow(images[i][0], cmap='gray', interpolation="none")
    if example_targets is not None:
      ax.text(0, 0.8, f"GT: {str(example_targets[i].item())}", transform=ax.transAxes, color="yellow")

  plt.show()

display_digits_grid(example_data, grid_size=(4, 10), example_targets=example_targets)

In [ ]:
class DenseLayer(nn.Module):
    def __init__(self, input_size, output_size, activation_f=nn.functional.elu):
      super(DenseLayer, self).__init__()
      self.fc = nn.Linear(input_size, output_size)
      self.activation_f = activation_f
      self.bn = nn.BatchNorm1d(output_size)

    def forward(self, x):
      x = self.fc(x)
      x = self.bn(x)
      if self.activation_f:
        x = self.activation_f(x)
      return x

class VAE(nn.Module):
    def __init__(self, input_size, n_outputs):
      super(VAE, self).__init__()
      d1_size = 400
      d2_size = 400
      latent_size = 20
      self.hidden1 = DenseLayer(input_size, d1_size, activation_f=nn.functional.relu)      
      self.hidden2 = DenseLayer(d1_size, d2_size, activation_f=nn.functional.relu)
      self.hidden3_mean = DenseLayer(d1_size, latent_size, None)
      self.hidden3_gamma = DenseLayer(d1_size, latent_size, None)
      self.hidden4 = DenseLayer(latent_size, d2_size, activation_f=nn.functional.relu)
      self.hidden5 = DenseLayer(d2_size, d1_size, activation_f=nn.functional.relu)
      self.output = DenseLayer(d1_size, n_outputs, None)

    def encode(self, x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        mu = self.hidden3_mean(x)
        gamma = self.hidden3_gamma(x)
        return mu, gamma

    def reparametrize(self, mu, gamma):
        std = torch.exp(0.5*gamma)
        noise = torch.randn_like(std)
        return mu + std * noise

    def decode(self, x):
        x = self.hidden4(x)
        x = self.hidden5(x)
        logits = self.output(x)
        return logits, F.sigmoid(logits)

    def forward(self, x):
        x_reshaped = x.view(x.shape[0], -1)
        mu, gamma = self.encode(x_reshaped)
        z = self.reparametrize(mu, gamma)
        # print(z.shape, z)
        # sys.exit(1)
        logits, sigmoid = self.decode(z)
        return logits, sigmoid, mu, gamma


In [ ]:
device = torch.device("cuda:0")
print(device)
model = VAE(28 * 28, 28 * 28)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses = []
train_counter = []
log_interval = 100

def vae_loss_function(recon_x, x, mu, gamma):
  BCE_reconstruction_loss = F.binary_cross_entropy(recon_x, x, reduction='sum')
  KLD_latent_loss = -0.5 * torch.sum(1 + gamma - mu.pow(2) - gamma.exp())

  return BCE_reconstruction_loss + KLD_latent_loss

torch.set_printoptions(threshold=10_000)
torch.set_printoptions(profile="full")

def train(epoch):
  model.train()
  train_loss = 0.0
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    data=data.to(device)
    logits, sigmoid_outputs, hidden_mean, hidden_gamma = model(data)
    # print("here", logits.shape, sigmoid_outputs.shape)

    # print(data.shape)
    # print(data.view(data.shape[0], -1))

    loss = vae_loss_function(sigmoid_outputs, data.view(data.shape[0],-1), hidden_gamma, hidden_gamma)
    loss.backward()
    train_losses.append(loss.item())
    train_loss += loss.item()
    optimizer.step()

    if batch_idx % log_interval == 0:
      print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({(100. * batch_idx / len(train_loader)):.0f}%)]\tLoss: {loss.item():.6f}')
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(train_loader.dataset)))


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs=2
for epoch in range(1, epochs + 1):
  train(epoch)

In [ ]:
n_digits = 20
codings_rnd = torch.randn((n_digits,20), dtype=torch.float32, device=device)
gen_numbers, gen_numbers_sig = model.decode(codings_rnd)
gen_numbers = gen_numbers_sig.reshape(n_digits, 1, 28, 28)
display_digits_grid(gen_numbers.detach().cpu().numpy(), grid_size=(2, 10), example_targets=None)